In [ ]:
import sys
import platform
import threading
from ctypes import POINTER, cast
from functools import wraps
from tkinter import Tk, Label, Scale, Button, HORIZONTAL, StringVar, N, S, E, W

try:
    if platform.system() != "Windows":
        raise SystemExit("This script runs only on Windows.")

    from comtypes import CLSCTX_ALL, CoInitialize, CoUninitialize, GUID
    from comtypes.client import CreateObject
    from pycaw.pycaw import IAudioEndpointVolume, IMMDeviceEnumerator

except ImportError as e:
    print("Error: Missing pycaw or comtypes. Did you run 'pip install pycaw comtypes'?", file=sys.stderr)
    raise SystemExit(f"Required COM/Windows component import failed: {e}")

eRender = 0
eMultimedia = 1

def ensure_com(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        CoInitialize()
        try:
            return func(*args, **kwargs)
        except Exception:
            import traceback
            print("Exception in volume handler:", file=sys.stderr)
            traceback.print_exc()
        finally:
            try:
                CoUninitialize()
            except Exception:
                pass
    return wrapper

def _create_mmdevice_enumerator():
    try:
        return CreateObject("MMDeviceEnumerator.MMDeviceEnumerator", interface=IMMDeviceEnumerator)
    except Exception:
        clsid = GUID("{BCDE0395-E52F-467C-8E3D-C4579291692E}")
        return CreateObject(clsid, interface=IMMDeviceEnumerator)

def _get_volume_interface_for_default():
    enumerator = _create_mmdevice_enumerator()
    default_device = enumerator.GetDefaultAudioEndpoint(eRender, eMultimedia)
    iface = default_device.Activate(IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
    return cast(iface, POINTER(IAudioEndpointVolume))

def _percent_to_scalar(p):
    return max(0.0, min(1.0, p / 100.0))

def _scalar_to_percent(s):
    return max(0.0, min(100.0, s * 100.0))

class VolumeController:
    def __init__(self, root, volume_var, mute_text_var):
        self.root = root
        self.volume_var = volume_var
        self.mute_text_var = mute_text_var
        self.is_muted = False
        
        CoInitialize()
        try:
            self.update_gui_from_system()
        except Exception as e:
            print("Warning: Could not read initial system state.", e)
        finally:
            CoUninitialize()

    @ensure_com
    def update_gui_from_system(self):
        vol = _get_volume_interface_for_default()
        
        cur_scalar = float(vol.GetMasterVolumeLevelScalar())
        cur_pct = round(_scalar_to_percent(cur_scalar))
        self.volume_var.set(cur_pct)
        
        self.is_muted = bool(vol.GetMute())
        self._update_mute_button_text()

    def _update_mute_button_text(self):
        if self.is_muted:
            self.mute_text_var.set("UNMUTE (Muted)")
        else:
            self.mute_text_var.set("MUTE (Active)")

    @ensure_com
    def set_volume_from_slider(self, new_pct_str):
        new_pct = int(float(new_pct_str))
        vol = _get_volume_interface_for_default()
        
        vol.SetMasterVolumeLevelScalar(_percent_to_scalar(new_pct), None)
        print(f"Volume set to: {new_pct}%")

        if new_pct > 0 and self.is_muted:
            vol.SetMute(0, None)
            self.is_muted = False
            self._update_mute_button_text()

    @ensure_com
    def toggle_mute_state(self):
        vol = _get_volume_interface_for_default()
        
        self.is_muted = not self.is_muted
        vol.SetMute(1 if self.is_muted else 0, None)
        
        self._update_mute_button_text()
        print(f"Mute toggled: {self.is_muted}")

def create_gui():
    root = Tk()
    root.title("System Volume Control")
    root.resizable(False, False)

    volume_value = StringVar()
    mute_button_text = StringVar()

    controller = VolumeController(root, volume_value, mute_button_text)
    
    Label(root, text="System Master Volume:", font=('Arial', 12, 'bold')).grid(row=0, column=0, columnspan=2, pady=10, sticky=W+E)

    volume_scale = Scale(
        root,
        from_=0,
        to=100,
        orient=HORIZONTAL,
        length=300,
        resolution=1,
        variable=volume_value,
        command=controller.set_volume_from_slider
    )
    volume_scale.grid(row=1, column=0, columnspan=2, padx=20, pady=5)

    mute_button = Button(
        root,
        textvariable=mute_button_text,
        command=controller.toggle_mute_state,
        bg="#FF6666" if controller.is_muted else "#66FF66",
        activebackground="#DD4444" if controller.is_muted else "#44DD44",
        font=('Arial', 10, 'bold')
    )
    mute_button.grid(row=2, column=0, columnspan=2, padx=20, pady=15, sticky=W+E)

    quit_button = Button(
        root,
        text="Quit",
        command=root.quit,
        font=('Arial', 10)
    )
    quit_button.grid(row=3, column=0, columnspan=2, padx=20, pady=(0, 10), sticky=W+E)
    
    root.mainloop()

if __name__ == "__main__":
    create_gui()

In [7]:
#SYSTEM MICROPHONE CONTROL
import sys
import platform
import threading
from ctypes import POINTER, cast
from functools import wraps
from tkinter import Tk, Label, Scale, Button, HORIZONTAL, StringVar, W, E, PhotoImage, Toplevel
import os

try:
    if platform.system() != "Windows":
        raise SystemExit("This script runs only on Windows.")

    from comtypes import CLSCTX_ALL, CoInitialize, CoUninitialize, GUID
    from comtypes.client import CreateObject
    from pycaw.pycaw import IAudioEndpointVolume, IMMDeviceEnumerator

except ImportError as e:
    print("Error: Missing pycaw or comtypes. Did you run 'pip install pycaw comtypes'?", file=sys.stderr)
    raise SystemExit(f"Required COM/Windows component import failed: {e}")

eCapture = 1
eMultimedia = 1

def ensure_com(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        CoInitialize()
        try:
            return func(*args, **kwargs)
        except Exception:
            import traceback
            print("Exception in microphone handler:", file=sys.stderr)
            traceback.print_exc()
        finally:
            try:
                CoUninitialize()
            except Exception:
                pass
    return wrapper

def _create_mmdevice_enumerator():
    try:
        return CreateObject("MMDeviceEnumerator.MMDeviceEnumerator", interface=IMMDeviceEnumerator)
    except Exception:
        clsid = GUID("{BCDE0395-E52F-467C-8E3D-C4579291692E}")
        return CreateObject(clsid, interface=IMMDeviceEnumerator)

def _get_volume_interface_for_default():
    enumerator = _create_mmdevice_enumerator()
    default_device = enumerator.GetDefaultAudioEndpoint(eCapture, eMultimedia)
    iface = default_device.Activate(IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
    return cast(iface, POINTER(IAudioEndpointVolume))

def _percent_to_scalar(p):
    return max(0.0, min(1.0, p / 100.0))

def _scalar_to_percent(s):
    return max(0.0, min(100.0, s * 100.0))

class MicVolumeController:
    def __init__(self, root, volume_var, mute_text_var):
        self.root = root
        self.volume_var = volume_var
        self.mute_text_var = mute_text_var
        self.is_muted = False
        
        CoInitialize()
        try:
            self.update_gui_from_system()
        except Exception as e:
            print("Warning: Could not read initial microphone state. Is a microphone connected?", e)
        finally:
            CoUninitialize()

    @ensure_com
    def update_gui_from_system(self):
        vol = _get_volume_interface_for_default()
        
        cur_scalar = float(vol.GetMasterVolumeLevelScalar())
        cur_pct = round(_scalar_to_percent(cur_scalar))
        self.volume_var.set(cur_pct)
        
        self.is_muted = bool(vol.GetMute())
        self._update_mute_button_text()

    def _update_mute_button_text(self):
        if self.is_muted:
            self.mute_text_var.set("UNMUTE (Muted)")
        else:
            self.mute_text_var.set("MUTE (Active)")
            
    @ensure_com
    def set_volume_from_slider(self, new_pct_str):
        new_pct = int(float(new_pct_str))
        vol = _get_volume_interface_for_default()
        
        vol.SetMasterVolumeLevelScalar(_percent_to_scalar(new_pct), None)
        print(f"Microphone Volume set to: {new_pct}%")

        if new_pct > 0 and self.is_muted:
            vol.SetMute(0, None)
            self.is_muted = False
            self._update_mute_button_text()

    @ensure_com
    def toggle_mute_state(self):
        vol = _get_volume_interface_for_default()
        
        self.is_muted = not self.is_muted
        vol.SetMute(1 if self.is_muted else 0, None)
        
        self._update_mute_button_text()
        print(f"Microphone Mute toggled: {self.is_muted}")
        
        mute_button = self.root.nametowidget(".!button")
        if self.is_muted:
            mute_button.config(bg="#FF6666", activebackground="#DD4444")
        else:
            mute_button.config(bg="#66FF66", activebackground="#44DD44")

def create_gui():
    root = Tk()
    root.title("Microphone Volume Control")
    root.resizable(False, False)

    volume_value = StringVar()
    mute_button_text = StringVar()

    controller = MicVolumeController(root, volume_value, mute_button_text)
    
    root.grid_columnconfigure(0, weight=1)
    root.grid_columnconfigure(1, weight=1)

    Label(root, text="Microphone Control", font=('Arial', 12, 'bold')).grid(row=0, column=0, columnspan=2, pady=10, padx=20, sticky=W+E)

    volume_scale = Scale(
        root, 
        from_=0, 
        to=100, 
        orient=HORIZONTAL, 
        length=300, 
        resolution=1,
        variable=volume_value,
        command=controller.set_volume_from_slider,
        highlightbackground="#CCCCCC",
        troughcolor="#E0F7FA"
    )
    volume_scale.grid(row=1, column=0, columnspan=2, padx=20, pady=5)

    mute_button = Button(
        root, 
        textvariable=mute_button_text, 
        command=controller.toggle_mute_state,
        bg="#FF6666" if controller.is_muted else "#66FF66",
        activebackground="#DD4444" if controller.is_muted else "#44DD44",
        fg="white",
        relief="raised",
        font=('Arial', 10, 'bold')
    )
    mute_button.grid(row=2, column=0, columnspan=2, padx=20, pady=15, sticky=W+E)

    quit_button = Button(
        root, 
        text="Quit", 
        command=root.quit,
        font=('Arial', 10),
        bg="#C0C0C0"
    )
    quit_button.grid(row=3, column=0, columnspan=2, padx=20, pady=(0, 10), sticky=W+E)
    
    root.mainloop()

if __name__ == "__main__":
    create_gui()

Microphone Volume set to: 50%
Microphone Volume set to: 62%
Microphone Volume set to: 63%
Microphone Volume set to: 64%
Microphone Volume set to: 65%
Microphone Volume set to: 66%
Microphone Volume set to: 67%
Microphone Volume set to: 68%
Microphone Volume set to: 69%
Microphone Volume set to: 70%
Microphone Volume set to: 71%
Microphone Volume set to: 72%
Microphone Volume set to: 73%
Microphone Volume set to: 75%
Microphone Volume set to: 76%
Microphone Volume set to: 78%
Microphone Volume set to: 79%
Microphone Volume set to: 81%
Microphone Volume set to: 82%
Microphone Volume set to: 83%
Microphone Volume set to: 84%
Microphone Volume set to: 85%
Microphone Volume set to: 86%
Microphone Volume set to: 87%
Microphone Volume set to: 88%
Microphone Volume set to: 89%
Microphone Volume set to: 90%
Microphone Volume set to: 91%
Microphone Volume set to: 92%
Microphone Volume set to: 93%
Microphone Volume set to: 94%
Microphone Volume set to: 95%
Microphone Volume set to: 96%
Microphone